# 05-2. 교차 검증과 그리드 서치 (p.242)
- 검증 세트 

In [1]:
import pandas as pd

wine = pd.read_csv('http://bit.ly/wine_csv_data')
# 교재 (p.242)에 적힌 링크에서 데이터를 바로 가져오기

wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [2]:
data = wine[['alcohol', 'sugar', 'pH']]
target = wine[['class']]

In [3]:
# 훈련 세트와 정답 세트를 먼저 나누겠다. 
# 내가 원하는 비율을 지정해서 훈련 세트와 정답 세트의 비율을 나눌 수도 있다. (p.243)

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = \
train_test_split( data, target)

# 전체 데이터를 4개로 나눠놨음.



In [4]:

# 이제 다시 그 나눈 것에 검증용 데이터를 추가해서 나누겠다. 
sub_input, val_input, sub_target, val_target = \
train_test_split(train_input, train_target)

# 그래서 검증용 데이터까지 총 6개로 나눔. 

In [5]:
test_input.shape

(1625, 3)

In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9980842911877394
0.8374384236453202


In [7]:
print(dt.score(test_input, test_target))

0.8492307692307692


#### 위와 같이 검증 세트를 나누는 과정이 복잡하다. 
> `교차검증` 을 이용항여 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용하자.
- 사이킷런에 `cross_validate()` 함수를 통해서, 교차검증을 실행하자

---

In [8]:
 # 교차검증
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
# train_input은 검증 데이터를 나누기 전
print(scores)

{'fit_time': array([0.00526953, 0.00518346, 0.00476623, 0.00474644, 0.00478458]), 'score_time': array([0.0017736 , 0.00167632, 0.0015409 , 0.00149536, 0.00146747]), 'test_score': array([0.84102564, 0.85538462, 0.85420945, 0.85215606, 0.84599589])}


#### `cross_validate()` 함수는 기본적으로 5폴드 교차검정을 실행한다. (p.246)

 #### 하이퍼파라미터 튜닝 (p.248)
> 사람의 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술이 있다. (e.g. `AutoML`)
> 데이터에 따라 하이퍼파라미터를 내가 직접 알맞게 조작하기가 쉽지 않다.
>
>  `그리드서치` : GridSearchGV => 튜닝을 자동화 

In [23]:
from sklearn.model_selection import GridSearchCV

params = {
    'min_impurity_decrease': [0.001, 0.002, 0.003, 0.004]
}
# 내가 테스트하고 싶은 파라미터를 다 적어놓는다. 
gs = GridSearchCV(dt, params, n_jobs=1)
# ✅ n_jobs는 GridSearchCV 또는 다른 병렬 처리 기능에서 CPU의 사용량(프로세스 개수)을 조절하는 역할을 합니다.

In [24]:
 # 위를 통해서 학습을 시키자.

gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(min_impurity_decrease=0.001),
             n_jobs=1,
             param_grid={'min_impurity_decrease': [0.001, 0.002, 0.003, 0.004]})

In [25]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.8696633825944171


In [26]:
print(gs.best_params_)

{'min_impurity_decrease': 0.001}
